<img src="assets/logo.png" width="800">

Made by **Balázs Nagy** and **Márk Domokos**

[<img src="assets/open_button.png">](https://colab.research.google.com/github/Fortuz/edu_Adaptive/blob/main/practices/L02%20-%20Multivariate%20Linear%20Regression_solved.ipynb)

# Labor 02: Multivariate linear regression

### Property prices:

In this exercise, we will extend our univariate linear regression model introduced in lab L01 to the multivariate linear regression case to estimate house prices.

Suppose we want to sell a house, but we want to know the real value of the house so that we do not lose money on the sale. One possible way to do this is to collect data and then estimate the real estate market price of the house by building a model based on the data. Our data will be the area of the property ($m^2$) and the number of rooms (units), and the price ($) determined at the time of sale.

### 1: Import the packages
We will need:
- NumPy for array management
- MatPlotLib pyplot package for visualization
- Pandas for data reading

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 2: Load data

Load in the data! Use the Pandas package to do this and then convert it into a numpy array.

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Fortuz/edu_Adaptive/main/practices/assets/Lab02/Lab2data.txt',header = None).to_numpy()   
X = data[:,0:2]                                                # sort X
m = X.shape[0]                                                 # number of samples
Y = data[:,2].reshape(m,1)                                     # sort Y

print('X:',X.shape)                                            # check the shapes
print('Y:',Y.shape)
print('Number of samples: ',m)
print(X)

### 3: Normalising values (Feature scaling & Mean normalization)

The features of the data points may be in different magnitudes. In our case, it is easy to see that there is at least an order of magnitude difference between the area of the property and the number of rooms. In such cases, it is worth normalising our values so that they fall within the same order of magnitude and all input variables fall within the range of [-1..1] or [0..1] interval. This operation will promote convergence, as there will be no dominant variable present to suppress the effect of other variables. <br>
For this we will use the following relationship:

$ x = \frac{x - mean(x)}{std(x)} $

, with other words the mean of the samples is subtracted from a given sample (mean normalization) and divided by the standard deviation of the samples (feature scaling).

Graphically imagining around the origin makes it easier to find the line that covers our data according to our hypothesis. It is therefore useful to trasform our data into this region.

<img src="assets/Lab02/Pics/L02_Scaling.png" width="350">

Create the normalizing function!

In [ ]:
def featureNormalize(X):
######################################################    
    avg    = np.mean(X,axis =0)                          # calculation of the average per column
    sigma  = np.std(X,axis = 0, ddof = 1)                # calculation of the standard deviation per column (corrected empirical)
    X_norm = (X-avg)/sigma 
######################################################
    return X_norm, avg, sigma                            # return the formula based result 

print('Normalizing X vector ...')                       
X_norm,avg,sigma = featureNormalize(X)                   # normalization
X_norm=np.column_stack((np.ones(m),X_norm))              # add bias

After normalization, the BIAS is added to the input X matrix.

### 4: Gradient based method
Following the example of the previous exercise, we create the gradient method in multivariable form! Our data structure is as follows.

<img src="assets/Lab02/Pics/L02_Matrixok.png" width="500">

Our hypothesis function for the multivariate case can be written as follows:

$ h_{w}(x)=w_0x_0+w_1x_1+w_2x_2+ ... +w_nx_n $ <br>

And with matrix operations:

<img src="assets/Lab02/Pics/L02_XW.png" width="550">

Formula of the cost function: <br>

$ C(W)=C(w_0,w_1,...,w_n)=\frac{1}{2m}\sum_{i=1}^{m}(h_w(x^i)-y^i)^2 $

Tip: When programming, take advantage of

<img src="assets/Lab02/Pics/L02_Sum.png" width="550">

General weight update formula of the gradient method:

$ \color{red}{(j=0...n)}\hspace{7mm} w_j:=w_j-\mu\frac{1}{2m}\sum_{i=1}^{m}((h_w(x^i)-y^i)\cdot x_j^i) $

$\color{red}{Pay\ attention\ to\ the\ simultaneous\ update!}$

In [ ]:
# Cost function
def computeCostMulti(X,Y,W):
#############################################     
    C=0
    C=(1/(2*m))*((X@W-Y).T)@(X@W-Y) 
############################################# 
    return C

In [ ]:
# Gradient descent Method
def gradientDescentMulti(X,Y,W,lr,epochs):              
#############################################    
    C_history = np.zeros((epochs,1))
    temp= np.zeros((np.size(W,0),1))

    for iteration in range(0,epochs):
       for i in range(0,np.size(W,0)):
           temp[i] = W[i]-(lr/(2*m))*np.sum((X@W-Y)*(X[:,i].reshape(m,1)), axis=None)
       W=temp
       C_history[iteration]=computeCostMulti(X,Y,W)
############################################# 
    return W, C_history

In [ ]:
print('Running gradient descent ...')
lr = 0.015                                                  # learning rate
epochs = 1200                                               # number of epochs
W=np.zeros((3,1))                                           # initial weights (0;0;0)
W,C_history= gradientDescentMulti(X_norm,Y,W,lr,epochs)     # Use the Gradient Descent Method
print('''Weights expected from gradient descent (approx.):
 [[340372.05039403]
 [109434.51046856]
 [ -5454.97874429 ]]
''')
print('Weights computed from gradient descent:\n', W)

if int(W[0]) == 340372 and int(W[1]) == 109434 and int(W[2]) == -5454:
    print("\n The gradientDescentMulti() function is good. You can proceed.")
else:
    print("\n Something not right. Please modify the function!")

Check the convergence with the help of a graph.

In [ ]:
plt.plot(C_history)                                                                 # C_history plot
plt.title("Gradient descent algorithm's effect through the iterations",pad= 20)
plt.xlabel("Iterations")
plt.ylabel("Cost function value")
plt.show()

### 5: Prediction
Let's estimate the price of a 1650 $m^2$, 3 bedroom property! Watch for the normalization of the data, which is also neccessary here.

In [ ]:
def predict(FEET, BED):
#############################################    
    price = (np.array([1, ((FEET-avg[0])/sigma[0]), (BED-avg[1])/sigma[1]]))@W
############################################# 
    return price

In [ ]:
FEET = 1650
BED = 3
price = predict(FEET, BED)
print('''Prediction for a 1650 sq-ft / 3 bedroom house:
(predicted price should be approx. $293000) %.2f''' % price)

## In a slightly different way: with high level packages

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

data = pd.read_csv('https://raw.githubusercontent.com/Fortuz/edu_Adaptive/main/practices/assets/Lab02/Lab2data.txt',header = None)   

X = data.iloc[:, 0:2].values.reshape(-1,2)                              # arrange X
Y = data.iloc[:, 2].values.reshape(-1,1)                                # arrange Y 

lin_reg = LinearRegression()                                            # creation of a linear regression model class
lin_reg.fit(X,Y)                                                        # fit based on X,Y

pred = lin_reg.predict([[1650,3]])                                      # prediction for a house with 1650 m^2 and 3 bedrooms
print('Prediction for a 1650 sq-ft / 3 bedroom house:\n %.2f' % pred)

<div style="text-align: right">This lab exercise uses elements from Andrew Ng's Machine Learning course.</div>